In [71]:
# Importing libraries
import pandas as pd
import random

# Read csv file into a pandas dataframe
df = pd.read_csv("books.csv")
df = df[df.language_code =='eng']
df['Title'] = df['title'].str.split('(').str[0]
    
def recommendation_random():
    df_new= df.sample(replace=True)[['Title', 'authors',"rating"]]
    print("Title: ", df_new[['Title']].to_string(index=False, header=False))
    print("Author: ", df_new[['authors']].to_string(index=False, header=False))
    print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
    return "Enjoy!"
def recommendation_rating():
    try:
        search=input('Enter the rating as: rating <rating>\n')
        if len(search.split())==2 and type(int(search.split()[1]))==int:
            search=search.split()
            rate=float(search[1])
            df_rate = df[df['rating'].astype(float) >= rate]
            df_new= df_rate.sample(replace=True)[['Title', 'authors',"rating"]]
            print("Title: ", df_new[['Title']].to_string(index=False, header=False))
            print("Author: ", df_new[['authors']].to_string(index=False, header=False))
            print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
            return "Enjoy!"
        else:
            return 'An error was encountered. Please use the proper format and try again.'
    except ValueError:
        return 'An error was encountered. Please use the proper format and try again.'

def author_search():
    author=input("Enter author name for a book suggestion ")
    authl=[]
    til=[]
    res_til=[]
    ratl=[]
    res_ratl=[]
    aui=adic['authors'].items()
    tii=adic['Title'].items()
    rai=adic['rating'].items()
    for i in aui:
        authl.append(i[1])
    for i in tii:
        til.append(i[1])
    for i in rai:
        ratl.append(i[1])
    for i in range(len(authl)):
        if author in authl[i]:
            res_til.append(til[i])
            res_ratl.append(ratl[i])
    title=random.choices(res_til)[0]
    i=res_til.index(title)
    rating=res_ratl[i]
    print('Title: '+title+'\nRating: '+rating)
    return "Enjoy your book!"

status=input('Welcome user! Do you wish to begin? (Y/N)\n')
while status=='Y'or status=='y':
    inp=input("Are you:\n1.Looking for book suggestions\n2.Looking for more books by an author\n")
    if inp=='1':
        subin=input('Do you want:\n1.Random recommendations\n2.Recommendations of a certain rating\n')
        if subin=='1':
            print(recommendation_random())
        if subin=='2':
            print(recommendation_rating())
    elif inp=='2':
        print(author_search())

    status=input('Hello! Do you wish to continue? (Y/N)\n')
else:
    print("Thank you for using our recommendation system!")

Welcome user! Do you wish to begin? (Y/N)
y
Are you:
1.Looking for book suggestions
2.Looking for more books by an author
2
Enter author name for a book suggestion Eoin Colfer
Title: Artemis Fowl 
Rating: 3.84
Enjoy your book!
Hello! Do you wish to continue? (Y/N)
y
Are you:
1.Looking for book suggestions
2.Looking for more books by an author
2
Enter author name for a book suggestion J.K. Rowling
Title: Harry Potter and the Sorcerer's Stone 
Rating: 4.47
Enjoy your book!
Hello! Do you wish to continue? (Y/N)
y
Are you:
1.Looking for book suggestions
2.Looking for more books by an author
2
Enter author name for a book suggestion J.K. Rowling
Title: Harry Potter and the Prisoner of Azkaban 
Rating: 4.56
Enjoy your book!
Hello! Do you wish to continue? (Y/N)
n
Thank you for using our recommendation system!
